# Epoch 100

In [ ]:
!pip install pandas
!pip install datasets

#press ESC to enter command mode

In [1]:
import os
import re
import glob
import shutil
import string
import pathlib

# set the data directory
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','data'))

#give matplotlib a folder to save its configs
os.environ['MPLCONFIGDIR'] = os.path.join(data_dir,'plt_configs')
import matplotlib.pyplot as plt

#give huggingface a folder to save its stuff in too
#you only need this if you are using a huggingface dataset
os.environ['HF_HOME'] = os.path.join(data_dir,'hf_cache')
import datasets

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.layers import TextVectorization

In [2]:
dataset = datasets.load_dataset("flxclxc/encoded_drug_reviews")

Using custom data configuration flxclxc--encoded_drug_reviews-ee0cdba36988e67d
Found cached dataset json (/tf/data/hf_cache/datasets/flxclxc___json/flxclxc--encoded_drug_reviews-ee0cdba36988e67d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame(dataset)

In [4]:
print(df)

                                                   train
0      {'patient_id': 184648, 'drugName': 'Efudex', '...
1      {'patient_id': 25268, 'drugName': 'Flector Pat...
2      {'patient_id': 172019, 'drugName': 'Amitiza', ...
3      {'patient_id': 196063, 'drugName': 'Stendra', ...
4      {'patient_id': 225264, 'drugName': 'Bupropion'...
...                                                  ...
53466  {'patient_id': 199190, 'drugName': 'Depo-Prove...
53467  {'patient_id': 188476, 'drugName': 'ParaGard',...
53468  {'patient_id': 105752, 'drugName': 'Methylpred...
53469  {'patient_id': 56713, 'drugName': 'Meclizine',...
53470  {'patient_id': 215006, 'drugName': 'Fluoxetine...

[53471 rows x 1 columns]


In [5]:
complete_dataset = dataset['train'].to_pandas()
complete_dataset.head() #this returns the top 5 rows

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,encoded
0,184648,Efudex,basal cell carcinoma,"""I have BCC on my upper arm and SCC on upper l...",1.0,"August 30, 2016",16,36,"[-0.0633561835, 0.0115883639, -0.0027463636, 0..."
1,25268,Flector Patch,pain,"""I tore my shoulder labrum and the pain can be...",8.0,"May 29, 2014",40,45,"[-0.083280459, 0.0182377025, 0.0619471855, 0.0..."
2,172019,Amitiza,irritable bowel syndrome,"""Amitiza is the best if you have ibs!""",10.0,"July 13, 2016",9,8,"[-0.0300639421, -0.0081300493, 0.0343461707, 0..."
3,196063,Stendra,erectile dysfunction,"""Viagra works in a strong, crude way with side...",10.0,"November 10, 2014",82,141,"[-0.0037669495, -0.0845683292, 0.0196341239, 0..."
4,225264,Bupropion,depression,"""I really wanted Wellbutrin to work. I was giv...",3.0,"October 4, 2015",15,62,"[-0.0633124188, 0.0167291258, 0.0707527027, 0...."


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53471 entries, 0 to 53470
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   53471 non-null  object
dtypes: object(1)
memory usage: 417.9+ KB
None


In [7]:
complete_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53471 entries, 0 to 53470
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     53471 non-null  int64  
 1   drugName       53471 non-null  object 
 2   condition      53471 non-null  object 
 3   review         53471 non-null  object 
 4   rating         53471 non-null  float64
 5   date           53471 non-null  object 
 6   usefulCount    53471 non-null  int64  
 7   review_length  53471 non-null  int64  
 8   encoded        53471 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 3.7+ MB


In [8]:
# unique values in column "drugName"
drugs = complete_dataset['drugName'].unique()
for drug in drugs:
    print(drug)

Efudex
Flector Patch
Amitiza
Stendra
Bupropion
Amiodarone
Sprintec
Acetaminophen / tramadol
Sertraline
Lisinopril
Levonorgestrel
Liraglutide
Fluoxetine
Methylprednisolone
Doxylamine / pyridoxine
Zoladex
Clarithromycin
Tranylcypromine
Phentermine
Prednisolone
Tarceva
Ezetimibe / simvastatin
Belsomra
Enbrel
Flexeril
Ethinyl estradiol / levonorgestrel
Prozac
Clomipramine
Klonopin
Ethinyl estradiol / norgestimate
Zovirax
Adipex-P
Vilazodone
Etonogestrel
Buspirone
Phentermine / topiramate
Micardis HCT
Dapsone
TriNessa
Estarylla
Venlafaxine
Aviane
Geodon
Tiotropium
Remicade
Synthroid
Minocycline
Ativan
Eletriptan
Levoxyl
Duloxetine
Dexbrompheniramine / pseudoephedrine
Loestrin 24 Fe
Haloperidol
Nexplanon
Robaxin
Cefdinir
Ortho Tri-Cyclen Lo
Desogen
Hiprex
Lorazepam
Loratadine / pseudoephedrine
Lyrica
Norco
Ethinyl estradiol / norelgestromin
Ambrisentan
Trulicity
Duac
Zyrtec
Propranolol
Alli
Exemestane
Mirena
NuvaRing
Ethinyl estradiol / norethindrone
Desogestrel / ethinyl estradiol
Magnesium

In [9]:
frequence = complete_dataset['drugName'].value_counts()
print(frequence)

Levonorgestrel                       1265
Etonogestrel                         1081
Ethinyl estradiol / norethindrone     869
Nexplanon                             736
Ethinyl estradiol / norgestimate      649
                                     ... 
Codimal DM                              1
Low-Ogestrel-21                         1
Crizotinib                              1
Pletal                                  1
Magnesium salicylate                    1
Name: drugName, Length: 2635, dtype: int64


In [10]:
# setup directories, create them if they do not exist within data folder

base_dir = os.path.join(data_dir,'drugs')
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

drug_names = ["Levonorgestrel",
              "Etonogestrel",
              "Ethinyl estradiol / norethindrone",
              "Nexplanon",
              "Ethinyl estradiol / norgestimate",
              "Ethinyl estradiol / levonorgestrel",
              "Phentermine",
              "Sertraline",
              "Escitalopram",
              "Mirena",
              "Implanon",
              "Gabapentin",
              "Miconazole",
              "Bupropion",
              "Venlafaxine",
              "Duloxetine",
              "Tramadol",
              "Clonazepam",
              "Citalopram",
              "Medroxyprogesterone",
              "Bupropion / naltrexone",
              "Varenicline",
              "Metronidazole",
              "Drospirenone / ethinyl estradiol",
              "Tioconazole",
              "Depo-Provera",
              "Liraglutide",
              "Skyla",
              "Fluoxetine",
              "Quetiapine",
              "Lo Loestrin Fe",
              "Alprazolam",
              "Chantix",
              "Amitriptyline",
              "Doxycycline",
              "Desvenlafaxine",
              "Trazodone",
              "Suprep Bowel Prep Kit",
              "Paroxetine",
              "NuvaRing",
              "Bisacodyl",
              "Lorcaserin"]
drug_directories = []

for drug_name in drug_names:
    current_drug_dir = os.path.join(data_dir,'drugs',drug_name)
    print(current_drug_dir)
    drug_directories.append(current_drug_dir)
    if not os.path.exists(current_drug_dir):
        os.makedirs(current_drug_dir)


/tf/data/drugs/Levonorgestrel
/tf/data/drugs/Etonogestrel
/tf/data/drugs/Ethinyl estradiol / norethindrone
/tf/data/drugs/Nexplanon
/tf/data/drugs/Ethinyl estradiol / norgestimate
/tf/data/drugs/Ethinyl estradiol / levonorgestrel
/tf/data/drugs/Phentermine
/tf/data/drugs/Sertraline
/tf/data/drugs/Escitalopram
/tf/data/drugs/Mirena
/tf/data/drugs/Implanon
/tf/data/drugs/Gabapentin
/tf/data/drugs/Miconazole
/tf/data/drugs/Bupropion
/tf/data/drugs/Venlafaxine
/tf/data/drugs/Duloxetine
/tf/data/drugs/Tramadol
/tf/data/drugs/Clonazepam
/tf/data/drugs/Citalopram
/tf/data/drugs/Medroxyprogesterone
/tf/data/drugs/Bupropion / naltrexone
/tf/data/drugs/Varenicline
/tf/data/drugs/Metronidazole
/tf/data/drugs/Drospirenone / ethinyl estradiol
/tf/data/drugs/Tioconazole
/tf/data/drugs/Depo-Provera
/tf/data/drugs/Liraglutide
/tf/data/drugs/Skyla
/tf/data/drugs/Fluoxetine
/tf/data/drugs/Quetiapine
/tf/data/drugs/Lo Loestrin Fe
/tf/data/drugs/Alprazolam
/tf/data/drugs/Chantix
/tf/data/drugs/Amitriptyli

In [11]:
data = np.array(["Levonorgestrel",
                 "Etonogestrel",
                 "Ethinyl estradiol / norethindrone",
                 "Nexplanon",
                 "Ethinyl estradiol / norgestimate",
                 "Ethinyl estradiol / levonorgestrel",
                 "Phentermine",
                 "Sertraline",
                 "Escitalopram",
                 "Mirena",
                 "Implanon",
                 "Gabapentin",
                 "Miconazole",
                 "Bupropion",
                 "Venlafaxine",
                 "Duloxetine",
                 "Tramadol",
                 "Clonazepam",
                 "Citalopram",
                 "Medroxyprogesterone",
                 "Bupropion / naltrexone",
                 "Varenicline",
                 "Metronidazole",
                 "Drospirenone / ethinyl estradiol",
                 "Tioconazole",
                 "Depo-Provera",
                 "Liraglutide",
                 "Skyla",
                 "Fluoxetine",
                 "Quetiapine",
                 "Lo Loestrin Fe",
                 "Alprazolam", 
                 "Chantix", 
                 "Amitriptyline",
                 "Doxycycline",
                 "Desvenlafaxine",
                 "Trazodone",
                 "Suprep Bowel Prep Kit",
                 "Paroxetine",
                 "NuvaRing",
                 "Bisacodyl",
                 "Lorcaserin"])
s = pd.Series(data)

In [12]:
print(s[:3])

0                       Levonorgestrel
1                         Etonogestrel
2    Ethinyl estradiol / norethindrone
dtype: object


In [13]:
print(s[-3:])

39      NuvaRing
40     Bisacodyl
41    Lorcaserin
dtype: object


In [14]:
complete_dataset.head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,encoded
0,184648,Efudex,basal cell carcinoma,"""I have BCC on my upper arm and SCC on upper l...",1.0,"August 30, 2016",16,36,"[-0.0633561835, 0.0115883639, -0.0027463636, 0..."
1,25268,Flector Patch,pain,"""I tore my shoulder labrum and the pain can be...",8.0,"May 29, 2014",40,45,"[-0.083280459, 0.0182377025, 0.0619471855, 0.0..."
2,172019,Amitiza,irritable bowel syndrome,"""Amitiza is the best if you have ibs!""",10.0,"July 13, 2016",9,8,"[-0.0300639421, -0.0081300493, 0.0343461707, 0..."
3,196063,Stendra,erectile dysfunction,"""Viagra works in a strong, crude way with side...",10.0,"November 10, 2014",82,141,"[-0.0037669495, -0.0845683292, 0.0196341239, 0..."
4,225264,Bupropion,depression,"""I really wanted Wellbutrin to work. I was giv...",3.0,"October 4, 2015",15,62,"[-0.0633124188, 0.0167291258, 0.0707527027, 0...."


In [15]:
drug_datasets = []
for drug_name in drug_names:

    filtered_df = complete_dataset[complete_dataset['drugName'] == drug_name]
    drug_datasets.append(filtered_df)

In [16]:
drug_datasets[0].head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,encoded
10,165698,Levonorgestrel,abnormal uterine bleeding,"""Got mirena December 2016 for heavy bleeding. ...",1.0,"April 14, 2017",11,40,"[0.0280154534, 0.0006723958, 0.0635507107, 0.0..."
15,129719,Levonorgestrel,abnormal uterine bleeding,"""I got the Mirena on April 23rd to help with m...",1.0,"June 9, 2015",12,152,"[-0.0308523662, -0.0149372043, 0.0436035134, 0..."
91,167552,Levonorgestrel,birth control,"""I'm 22, no kids and I just got the Skyla inse...",9.0,"February 5, 2016",1,111,"[-0.0444505177, -0.0698180422, 0.0684886724, 0..."
112,167211,Levonorgestrel,birth control,"""I had this IUD for about 5 months. I experien...",3.0,"May 3, 2016",0,109,"[-0.0279869325, -0.0604720227, 0.0309675075, -..."
163,165803,Levonorgestrel,birth control,"""I'm 23 no children. It didn't really hurt tha...",6.0,"March 21, 2017",2,120,"[-0.0320023857, -0.0279171392, 0.066063568, 0...."


In [17]:
drug_datasets[41].head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,encoded
191,76586,Lorcaserin,weight loss,"""So not worth it. I had all kinds of bad effec...",1.0,"July 3, 2017",14,144,"[0.0001945561, -0.008763249, 0.0430541486, 0.0..."
576,76918,Lorcaserin,weight loss,"""I'm Cheway69 who ever read my post , just a r...",5.0,"May 25, 2015",25,152,"[-0.0962474123, -0.0540102273, 0.0341046676, 0..."
804,76677,Lorcaserin,weight loss,"""I have been on this for two and a half weeks ...",10.0,"January 30, 2017",72,29,"[-0.0023870701, 0.0463704094, 0.0202984493, 0...."
856,77211,Lorcaserin,weight loss,"""I thought Belviq was the answer to my prayers...",2.0,"May 10, 2014",46,92,"[0.0004727186, -0.0455520265, 0.064966619, 0.0..."
1191,76949,Lorcaserin,weight loss,"""I read hundreds of reviews, talked to my doct...",1.0,"May 21, 2015",28,125,"[-0.0134229176, -0.0871194154, 0.057487864, 0...."


In [18]:
print(drug_names[41])
print(drug_directories[41])

Lorcaserin
/tf/data/drugs/Lorcaserin


In [19]:
drug_names
drug_directories
drug_datasets

for i in range(len(drug_names)):
    review_counter = 0
    for text in drug_datasets[i]['review']:
        with open(os.path.join(drug_directories[i],str(review_counter)+'.txt'), 'w') as f:
            f.write(text)
        review_counter+=1

# Loading data set for training

https://www.tensorflow.org/tutorials/load_data/text

In [20]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    pathlib.Path(base_dir),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 15744 files belonging to 40 classes.
Using 12596 files for training.


In [21]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print("Patient Review: ", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])

Patient Review:  b'"Wellbutrin is a good help with severe grief reaction without the need for prozac, etc. For me, it gave me a break from the grief. No great feelings of joy but less deep gutwrenching grief, but also less ability to cry. No weight gain, no real side-effects.  I\'ve taken it for about 3 months and it doesn\'t change my personality.  I take a low dose of about 250 mg / day and I\'m 165 lbs.  150mg wasn\'t enough."'
Label: 3
Patient Review:  b'"So, Saxenda was effective for me for weight loss but the side effects I cannot tolerate. \r\r\n\r\r\n I was SO tired I could not stay awake when I get home from work.  I actually started to think something serious might be wrong with me to be this fatigued. So I stopped Saxenda and then I felt normal again --so realized that Saxenda was making me so that fatigued, I realized I don\'t want to feel this way to lose weight.\r\r\n\r\r\nMy insurance did not cover either.  $$$$$$"'
Label: 20
Patient Review:  b'"I would not recommend thi

In [22]:
for i, label in enumerate(raw_train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to Alprazolam
Label 1 corresponds to Amitriptyline
Label 2 corresponds to Bisacodyl
Label 3 corresponds to Bupropion
Label 4 corresponds to Bupropion 
Label 5 corresponds to Chantix
Label 6 corresponds to Citalopram
Label 7 corresponds to Clonazepam
Label 8 corresponds to Depo-Provera
Label 9 corresponds to Desvenlafaxine
Label 10 corresponds to Doxycycline
Label 11 corresponds to Drospirenone 
Label 12 corresponds to Duloxetine
Label 13 corresponds to Escitalopram
Label 14 corresponds to Ethinyl estradiol 
Label 15 corresponds to Etonogestrel
Label 16 corresponds to Fluoxetine
Label 17 corresponds to Gabapentin
Label 18 corresponds to Implanon
Label 19 corresponds to Levonorgestrel
Label 20 corresponds to Liraglutide
Label 21 corresponds to Lo Loestrin Fe
Label 22 corresponds to Lorcaserin
Label 23 corresponds to Medroxyprogesterone
Label 24 corresponds to Metronidazole
Label 25 corresponds to Miconazole
Label 26 corresponds to Mirena
Label 27 corresponds to Nexpla

In [23]:
# Create a validation set.

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    pathlib.Path(base_dir),
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)


Found 15744 files belonging to 40 classes.
Using 3148 files for validation.


In [24]:
# Prepare dataset for training

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [25]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [26]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [27]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [28]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'"This drug has been a lifesaver to me. It\'s helped provide me with an antidepressant effect, whereas lithium only controlled the mania part of my mood. I use this in combination with other drugs and so far there hasn\'t been any side effects. Within 3 weeks the effects peaked. I would definitely recommend this drug. This has made a difference in my mood, the way I think, and my overall perspective. This improved my motivation to get work done, since I wouldn\'t be just sitting around reminiscing in a negative thought loop. It gives me mental freedom and allows me to escape brain fog that has imprisoned my thoughts. Seriously, don\'t be afraid to try this drug."', shape=(), dtype=string)
Label Bupropion
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  15,  216,   48,   24,    8, 1719,    5,   18,   41,  227, 3276,
          18,   21,   97,  927,  254, 2743, 2529,   52, 1698,    4, 2395,
         452,   12,    7,  121,    2,  232,   15,   1

In [29]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  yr
 313 --->  fine
Vocabulary size: 10000


In [30]:
#Dataset does not contain test so I'm omitting test_ds = raw_test_ds.map(vectorize_text)

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [31]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create the model

In [32]:
embedding_dim = 16

In [33]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(len(drug_names), activation='softmax')])


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 42)                714       
                                                                 
Total params: 160,730
Trainable params: 160,730
Non-trainable params: 0
__________________________________________________

# Train the model

In [35]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer='nadam',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [36]:
epochs = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/100
394/394 [==============================] - 1s 2ms/step - loss: 3.5751 - sparse_categorical_accuracy: 0.1287 - val_loss: 3.4490 - val_sparse_categorical_accuracy: 0.1290
Epoch 2/100
394/394 [==============================] - 1s 1ms/step - loss: 3.4292 - sparse_categorical_accuracy: 0.1351 - val_loss: 3.4165 - val_sparse_categorical_accuracy: 0.1290
Epoch 3/100
394/394 [==============================] - 1s 1ms/step - loss: 3.3955 - sparse_categorical_accuracy: 0.1354 - val_loss: 3.3845 - val_sparse_categorical_accuracy: 0.1290
Epoch 4/100
394/394 [==============================] - 1s 1ms/step - loss: 3.3580 - sparse_categorical_accuracy: 0.1358 - val_loss: 3.3445 - val_sparse_categorical_accuracy: 0.1290
Epoch 5/100
394/394 [==============================] - 1s 1ms/step - loss: 3.3094 - sparse_categorical_accuracy: 0.1375 - val_loss: 3.2924 - val_sparse_categorical_accuracy: 0.1293
Epoch 6/100
394/394 [==============================] - 1s 1ms/step - loss: 3.2489 - sparse_cate

394/394 [==============================] - 1s 1ms/step - loss: 1.7499 - sparse_categorical_accuracy: 0.4506 - val_loss: 1.7573 - val_sparse_categorical_accuracy: 0.4578
Epoch 47/100
394/394 [==============================] - 1s 1ms/step - loss: 1.7349 - sparse_categorical_accuracy: 0.4607 - val_loss: 1.7445 - val_sparse_categorical_accuracy: 0.4628
Epoch 48/100
394/394 [==============================] - 1s 1ms/step - loss: 1.7176 - sparse_categorical_accuracy: 0.4624 - val_loss: 1.7313 - val_sparse_categorical_accuracy: 0.4663
Epoch 49/100
394/394 [==============================] - 1s 2ms/step - loss: 1.7074 - sparse_categorical_accuracy: 0.4622 - val_loss: 1.7198 - val_sparse_categorical_accuracy: 0.4701
Epoch 50/100
394/394 [==============================] - 1s 1ms/step - loss: 1.6939 - sparse_categorical_accuracy: 0.4659 - val_loss: 1.7080 - val_sparse_categorical_accuracy: 0.4740
Epoch 51/100
394/394 [==============================] - 1s 1ms/step - loss: 1.6818 - sparse_categorical

394/394 [==============================] - 1s 1ms/step - loss: 1.3347 - sparse_categorical_accuracy: 0.5628 - val_loss: 1.3991 - val_sparse_categorical_accuracy: 0.5473
Epoch 92/100
394/394 [==============================] - 1s 1ms/step - loss: 1.3239 - sparse_categorical_accuracy: 0.5645 - val_loss: 1.3937 - val_sparse_categorical_accuracy: 0.5467
Epoch 93/100
394/394 [==============================] - 1s 1ms/step - loss: 1.3136 - sparse_categorical_accuracy: 0.5653 - val_loss: 1.3884 - val_sparse_categorical_accuracy: 0.5515
Epoch 94/100
394/394 [==============================] - 1s 1ms/step - loss: 1.3072 - sparse_categorical_accuracy: 0.5657 - val_loss: 1.3846 - val_sparse_categorical_accuracy: 0.5508
Epoch 95/100
394/394 [==============================] - 1s 1ms/step - loss: 1.3091 - sparse_categorical_accuracy: 0.5658 - val_loss: 1.3797 - val_sparse_categorical_accuracy: 0.5527
Epoch 96/100
394/394 [==============================] - 1s 1ms/step - loss: 1.3043 - sparse_categorical

In [37]:
loss, accuracy = model.evaluate(train_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

394/394 [==============================] - 0s 549us/step - loss: 1.1350 - sparse_categorical_accuracy: 0.6393
Loss:  1.1350092887878418
Accuracy:  0.6393299698829651


In [38]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

99/99 [==============================] - 0s 548us/step - loss: 1.3570 - sparse_categorical_accuracy: 0.5575
Loss:  1.357041358947754
Accuracy:  0.5574968457221985


In [39]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


,loss,sparse_categorical_accuracy,val_loss,val_sparse_categorical_accuracy,epoch
95,1.304265,0.566688,1.375901,0.554003,95
96,1.294080,0.574151,1.370524,0.556862,96
97,1.288654,0.575103,1.367072,0.555591,97
98,1.281025,0.574786,1.362046,0.558450,98
99,1.278394,0.579628,1.357041,0.557497,99


# Export the model

In [40]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('softmax')
])

export_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='nadam',
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)


In [41]:
loss, accuracy = export_model.evaluate(raw_train_ds)
print(accuracy)


394/394 [==============================] - 0s 934us/step - loss: 3.3320 - sparse_categorical_accuracy: 0.6393
0.6393299698829651


# Inference on new data

In [42]:
examples = [
  "This medication relieved constipation.",
  "This medication helped me quit smoking."
]

predictions=export_model.predict(examples)


1/1 [==============================] - 0s 78ms/step


In [43]:
for j in range(len(examples)):
    print(examples[j])
    prediction=predictions[j]
    for i, label in enumerate(raw_train_ds.class_names):
        print(label+':'+str(prediction[i]))

This medication relieved constipation.
Alprazolam:0.02384949
Amitriptyline:0.024281094
Bisacodyl:0.023895195
Bupropion:0.023954181
Bupropion :0.023673292
Chantix:0.023567444
Citalopram:0.023772689
Clonazepam:0.024100902
Depo-Provera:0.023418762
Desvenlafaxine:0.02369021
Doxycycline:0.024041044
Drospirenone :0.023408534
Duloxetine:0.0240467
Escitalopram:0.023802523
Ethinyl estradiol :0.024243739
Etonogestrel:0.023400104
Fluoxetine:0.023572661
Gabapentin:0.02465087
Implanon:0.02331523
Levonorgestrel:0.023706432
Liraglutide:0.023711707
Lo Loestrin Fe:0.023360679
Lorcaserin:0.023865284
Medroxyprogesterone:0.023455268
Metronidazole:0.024913482
Miconazole:0.024010234
Mirena:0.023417478
Nexplanon:0.02335426
NuvaRing:0.023401128
Paroxetine:0.023634298
Phentermine:0.024118079
Quetiapine:0.024328386
Sertraline:0.024304397
Skyla:0.023317024
Suprep Bowel Prep Kit:0.023619302
Tioconazole:0.023836892
Tramadol:0.0247627
Trazodone:0.02419129
Varenicline:0.023622982
Venlafaxine:0.023892093
This medicat